In [15]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0")


In [16]:
// Import statements
import java.time.Duration
import java.time.LocalDateTime
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.score.director.ScoreDirector
import ai.timefold.solver.core.api.domain.variable.VariableListener
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.variable.InverseRelationShadowVariable
import ai.timefold.solver.core.api.domain.variable.PlanningVariableGraphType
import ai.timefold.solver.core.api.domain.variable.ShadowVariable
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

In [17]:
// Skill
data class Skill(
    val id: String,
    val name: String
)

// Location
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun distanceTo(other: Location): Double {
        val xDiff = latitude - other.latitude
        val yDiff = longitude - other.longitude
        return kotlin.math.sqrt(xDiff * xDiff + yDiff * yDiff)
    }

    fun travelTimeTo(other: Location): Duration {
        val distanceKm = distanceTo(other) * 111  // Approximate conversion to kilometers
        val averageSpeedKmh = 50.0
        val timeInHours = distanceKm / averageSpeedKmh
        return Duration.ofMinutes((timeInHours * 60).toLong())
    }
}

// TimeWindow
data class TimeWindow(
    val start: LocalDateTime,
    val end: LocalDateTime
)


In [18]:
interface Standstill {
    val location: Location
    val departureTime: LocalDateTime?
}


In [19]:
@PlanningEntity
class Meeting(
    @PlanningId
    val id: String,
    override val location: Location,
    val requiredSkills: List<Skill>,
    val timeWindow: TimeWindow,
    val duration: Duration
) : Standstill {

    @PlanningVariable(
        valueRangeProviderRefs = ["standstillRange"],
        graphType = PlanningVariableGraphType.CHAINED
    )
    var previousStandstill: Standstill? = null

    @ShadowVariable(
        sourceVariableName = "previousStandstill"
    )
    var arrivalTime: LocalDateTime? = null

    @ShadowVariable(
        sourceVariableName = "arrivalTime"
    )
    override var departureTime: LocalDateTime? = null

    @InverseRelationShadowVariable(sourceVariableName = "previousStandstill")
    var nextMeeting: Meeting? = null
}


Line_19.jupyter.kts (18:9 - 19:6) No value passed for parameter 'variableListenerClass'
Line_19.jupyter.kts (23:9 - 24:6) No value passed for parameter 'variableListenerClass'

In [14]:
@PlanningEntity
class Meeting(
    @PlanningId
    val id: String,
    override val location: Location,
    val requiredSkills: List<Skill>,
    val timeWindow: TimeWindow,
    val duration: Duration
) : Standstill {

    // Planning variable: previous standstill (Agent or another Meeting)
    @PlanningVariable(
        valueRangeProviderRefs = ["standstillRange"],
        graphType = PlanningVariableGraphType.CHAINED
    )
    var previousStandstill: Standstill? = null

    // Shadow variables
    @ShadowVariable(
        variableListenerClass = ArrivalTimeUpdatingVariableListener::class,
        sourceVariableName = "previousStandstill"
    )
    var arrivalTime: LocalDateTime? = null

    @ShadowVariable(
        variableListenerClass = DepartureTimeUpdatingVariableListener::class,
        sourceVariableName = "arrivalTime"
    )
    override var departureTime: LocalDateTime? = null

    // For chaining
    @InverseRelationShadowVariable(sourceVariableName = "previousStandstill")
    var nextMeeting: Meeting? = null
}


Line_14.jupyter.kts (5:28 - 36) Type of 'location' is not a subtype of the overridden property 'public abstract val location: Line_7_jupyter.Location defined in Line_9_jupyter.Standstill'
Line_14.jupyter.kts (20:33 - 68) Unresolved reference: ArrivalTimeUpdatingVariableListener
Line_14.jupyter.kts (20:33 - 75) An annotation argument must be a compile-time constant
Line_14.jupyter.kts (26:33 - 70) Unresolved reference: DepartureTimeUpdatingVariableListener
Line_14.jupyter.kts (26:33 - 77) An annotation argument must be a compile-time constant